In [1]:
import traci
import sumolib

SUMO_BINARY = "sumo-gui"  # Use "sumo" for non-GUI
SUMO_CONFIG = "busStop.sumocfg"
NET_FILE = "osm.net.xml"  # Replace with actual network file

# Load SUMO network
net = sumolib.net.readNet(NET_FILE)

# Start SUMO
traci.start([SUMO_BINARY, "-c", SUMO_CONFIG,"--start"])

# Get all bus stops
bus_stops = traci.busstop.getIDList()
print(f"Total Bus Stops: {len(bus_stops)}")

# Store bus stop positions
bus_stop_positions = {
    stop: traci.lane.getShape(traci.busstop.getLaneID(stop))[0] for stop in bus_stops
}
print("Bus Stop Positions:", bus_stop_positions)



Total Bus Stops: 56
Bus Stop Positions: {'-2207611901': (279.5, 4207.29), '11975930340': (5203.1, 2423.19), '11982377441': (5118.34, 2195.38), '11982377442': (5214.21, 2411.83), '1702530312': (5096.08, 3043.36), '2141425962': (32.8, 3453.91), '2141425973': (41.48, 3450.94), '2207611901': (270.69, 4160.27), '5189265436': (1086.0, 1924.34), '5189265479': (4104.74, 3040.27), '5777572361': (1088.33, 1929.23), '5777572412': (63.05, 2056.98), '5777572424': (4081.6, 3032.02), '664440061': (504.91, 5718.42), '7468327498': (2315.09, 4979.42), '7628479190': (4170.54, 3082.84), '7976140061': (1088.33, 1929.23), '7976140062': (4104.74, 3040.27), '7976140064': (1088.33, 1929.23), '7976140065': (2316.7, 3947.97), '7976140067': (1751.32, 3581.04), '7976140068': (1088.33, 1929.23), '7976140070': (1088.33, 1929.23), '7976140071': (1751.32, 3581.04), '7n_tnk_down': (3686.52, 4814.81), '7n_tnk_up': (3678.26, 4800.32), 'amri_hos': (2389.08, 1741.24), 'bm_down_1': (2078.05, 953.97), 'bm_up_1': (2047.62, 96

In [2]:
import networkx as nx

# Create a graph from the SUMO network
G = nx.DiGraph()

# Add edges based on the SUMO road network
for edge in net.getEdges():
    from_node = edge.getFromNode().getID()
    to_node = edge.getToNode().getID()
    length = edge.getLength()
    G.add_edge(from_node, to_node, weight=length)

# Generate routes using shortest paths between bus stops


valid_routes = []
for i, stop1 in enumerate(bus_stops):
    for stop2 in bus_stops[i + 1 :]:  # Avoid duplicates
        lane1 = traci.busstop.getLaneID(stop1)
        lane2 = traci.busstop.getLaneID(stop2)

        edge1 = net.getLane(lane1).getEdge()
        edge2 = net.getLane(lane2).getEdge()
        
        try:
            shortest_path = nx.shortest_path(G, edge1.getFromNode().getID(), edge2.getToNode().getID(), weight="weight")
            valid_routes.append((stop1, stop2, shortest_path))
        except nx.NetworkXNoPath:
            continue  # Skip if no path exists

print(f"Generated {len(valid_routes)} valid routes.")
traci.close()

Generated 1250 valid routes.


In [3]:
import xml.etree.ElementTree as ET

def write_routes_xml(routes, output_file="valid_routes.rou.xml"):
    root = ET.Element("routes")

    for i, (stop1, stop2, path) in enumerate(routes):
        route_id = f"route_{i}"
        edges = " ".join(path)
        
        route = ET.SubElement(root, "route", id=route_id, edges=edges)
        
        # Add a bus vehicle type
        vehicle = ET.SubElement(root, "vehicle", id=f"bus_{i}", type="bus", route=route_id, depart="0")
    
    tree = ET.ElementTree(root)
    tree.write(output_file)

# Write the valid routes to an XML file
write_routes_xml(valid_routes)
print("Routes saved to valid_routes.rou.xml")


Routes saved to valid_routes.rou.xml
